# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import ast

In [ ]:
movies= pd.read_csv("tmdb_5000_movies.csv")
credit= pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head(1)

In [ ]:
credit.head(1)

In [ ]:
movies=movies.merge(credit, on="title")

In [ ]:
movies.head(1)

In [ ]:
movies.shape

In [ ]:
# genres, id, keywords, title, overview, cast, crew

movies=movies[["movie_id", "title","overview", "genres","keywords", "cast", "crew"]]

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.shape

# Data Preprocessing

In [ ]:
movies["genres"].values

In [ ]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    return L


In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

# ["Action","Adventure","Fantasy","Science Fi"]

movies["genres"]=movies["genres"].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies.iloc[0].keywords

In [ ]:
movies.iloc[0].genres

In [ ]:
movies["keywords"]=movies["keywords"].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies["cast"].values

In [ ]:
movies.iloc[0].cast

In [ ]:
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter < 3:
            L.append(i["name"])
            counter+=1
        else:
            break
    return L                       


In [ ]:
#movies["genres"].astype

In [ ]:
movies["cast"]=movies["cast"].apply(convert3)

In [ ]:
movies.head(1)

In [ ]:
movies["crew"][0]

In [ ]:
movies.iloc[0].crew

In [ ]:
movies["crew"].values

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            L.append(i["name"])
            break
    return L

In [ ]:
movies["crew"]=movies["crew"].apply(fetch_director)

In [ ]:
movies.head(1)      

In [ ]:
movies["overview"][0]

In [ ]:
movies.iloc[0].overview

In [ ]:
movies["overview"].values

In [ ]:
movies["overview"]=movies["overview"].apply(lambda x:x.split())

In [ ]:
movies.head(1)

In [ ]:
movies["overview"][0]

In [ ]:
def collapse(obj):
    L1 = []
    for i in obj:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movies["genres"] = movies["genres"].apply(collapse)
movies["keywords"] = movies["keywords"].apply(collapse)
movies["cast"] = movies["cast"].apply(collapse)
movies["crew"] = movies["crew"].apply(collapse)


In [ ]:
movies.head(1)

In [ ]:
movies["tags"]= movies["overview"]+ movies["genres"]+movies["keywords"] + movies["cast"] + movies["crew"]

In [ ]:
new_df= movies[["movie_id", "title","tags"]]

In [ ]:
new_df

In [ ]:
# Check if the element is a list (or any iterable) before joining
new_df["tags"] = new_df["tags"].apply(lambda x: " ".join(x))


In [ ]:
new_df.head()

In [ ]:
new_df["tags"][0]

In [ ]:
new_df["tags"] = new_df["tags"].apply(lambda x: x.lower())


In [ ]:
new_df

# TEXT VECTORIZATION

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv= CountVectorizer(max_features=5000, stop_words="english")

In [ ]:
vectors = cv.fit_transform(new_df["tags"]).toarray()


In [ ]:
vectors[0]

In [ ]:
vectors.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
similarity.shape

In [ ]:
index= new_df[new_df["title"]== "Avatar"].index[0]
index

In [ ]:
distances= sorted(list(enumerate(similarity[0])), reverse=True, key= lambda x: x[1])
distances

In [ ]:
def recommend(movie):
    index= new_df[new_df["title"]== movie].index[0]
    distances= sorted(list(enumerate(similarity[index])), reverse=True, key= lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend("Batman Begins")

# Deployment

In [ ]:
import pickle


In [ ]:
pickle.dump(new_df.to_dict(), open("movies_dict.pkl", "wb"))

In [ ]:
new_df.to_dict()

In [ ]:
pickle.dump(similarity, open("similarity.pkl","wb"))